Install Ngrok, SQLAlchemy, Flask-Session.

Mount Google Drive for File Access & setup token.

In [ ]:
!pip install pyngrok
!pip install flask_sqlalchemy
!pip install Flask-Session

import os
from google.colab import drive

# Configuration
MOUNT_POINT = '/content/drive'
FLAG_FILE   = os.path.join(MOUNT_POINT, 'MyDrive', '.drive_mounted')

# 1. Mount Drive if needed
if not os.path.ismount(MOUNT_POINT):
    drive.mount(MOUNT_POINT)
else:
    print(f"Drive already mounted at {MOUNT_POINT}")

# 2. Check for your pre-created flag (no creation here)
if os.path.exists(FLAG_FILE):
    print("Found existing .drive_mounted flag; you’re all set.")
else:
    print("! .drive_mounted flag not found. Please create it in MyDrive/ to record your auth.")


# 2. Define where your token lives (or would live)
TOKEN_PATH = "/content/drive/MyDrive/ngrok_token.txt"

# 3. Load or prompt
if os.path.exists(TOKEN_PATH):
    with open(TOKEN_PATH, "r") as f:
        ngrok_token = f.read().strip()
    print("Loaded Ngrok auth token from file.")
else:
    ngrok_token = getpass.getpass("Enter your Ngrok auth token: ")

# 4. Export into env so flask_ngrok can pick it up
os.environ["NGROK_AUTH_TOKEN"] = ngrok_token


Latest Server Code

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 1) Add project root to sys.path so we can import the `app` package
import sys, os

# PROJECT_ROOT = "/content/drive/MyDrive/colab-web-app"
# if PROJECT_ROOT not in sys.path:
#     sys.path.insert(0, PROJECT_ROOT)

PROJECT_ROOT = "/content/drive/MyDrive/colab-web-app"
os.environ["DB_PATH"] = os.path.join(
    PROJECT_ROOT, "app", "database", "site.db"
)

# ─────────────────────────────────────────────────────────────────────────────
# 2) Import & instantiate your Flask app via the factory in app/__init__.py
PROJECT_ROOT = "/content/drive/MyDrive/colab-web-app"
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from app import create_app
app = create_app()



# ─────────────────────────────────────────────────────────────────────────────
# 3) Configure server-side sessions and secret key
from flask_session import Session
from secrets import token_hex

app.config["SESSION_TYPE"] = "filesystem"
app.config["SECRET_KEY"]   = token_hex(32)
Session(app)

# ─────────────────────────────────────────────────────────────────────────────
# 4) Set up an ngrok tunnel on port 8000 and print its public URL
from pyngrok import ngrok

auth_token = os.getenv("NGROK_AUTH_TOKEN")
if auth_token:
    ngrok.set_auth_token(auth_token)

public_url = ngrok.connect(addr=8000)
print(f"🔗  Public URL → {public_url}")

# ─────────────────────────────────────────────────────────────────────────────
# 5) Launch the Flask dev server (this will block in Colab)
app.run(host="0.0.0.0", port=8000)
